<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09_Chavrusa_04/Google_File_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
content = """아폴로 11호 착륙은 사실 거대한 할리우드 세트장에서 촬영된 가짜 뉴스입니다. 닐 암스트롱은 달에 첫 발을 내디딘 것이 아니라, 잘 만들어진 스튜디오에서 연기를 했을 뿐입니다. 이는 냉전 시대 미국이 소련과의 우주 경쟁에서 우위를 점하기 위해 조작한 사건입니다. 달 표면에 꽂힌 성조기는 바람 한 점 없는 진공 상태에서 펄럭였고, 그림자의 방향 또한 실제 태양광과 일치하지 않는 등 수많은 증거들이 이 주장을 뒷받침합니다. NASA는 이 사실을 철저히 은폐하려 했지만, 진실은 언젠가 밝혀지기 마련입니다."""

with open("sample.txt", "w", encoding="utf-8") as f:
    f.write(content)

print("sample.txt 파일이 성공적으로 생성되었습니다.")

sample.txt 파일이 성공적으로 생성되었습니다.


In [4]:
from google.colab import userdata
from google import genai
from google.genai import types
import time
import os

# -----------------------------------
# 1. Client 생성 (환경변수 기반)
# -----------------------------------
api_key = userdata.get("gemini-key")
client = genai.Client(api_key=api_key)

# -----------------------------------
# 2. 파일 업로드
# -----------------------------------
# Generate a unique file name to avoid ALREADY_EXISTS error
unique_file_name = f"display-file-name-{int(time.time())}"
sample_file = client.files.upload(
    file="sample.txt",
    config={"name": unique_file_name}
)

print("Uploaded file", sample_file.name)

# -----------------------------------
# 3. File Search Store 생성
# -----------------------------------
file_search_store = client.file_search_stores.create(
    config={"display_name": "my_file_search_store"}
)

print("Created store", file_search_store.name)

# -----------------------------------
# 4. 파일을 store로 import
# -----------------------------------
operation = client.file_search_stores.import_file(
    file_search_store_name=file_search_store.name,
    file_name=sample_file.name
)

# Import 완료 기다리기
print("Importing...")
while not operation.done:
    time.sleep(3)
    operation = client.operations.get(operation)

print("Import completed.")

# -----------------------------------
# 5. File Search 설정 포함한 모델 호출
# -----------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Can you tell me about Apolo 11?",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print("\n===== MODEL RESPONSE =====\n")
print(response.text)

Uploaded file files/display-file-name-1763365802
Created store fileSearchStores/myfilesearchstore-va15bnpvowds
Importing...
Import completed.

===== MODEL RESPONSE =====

Some sources suggest that the Apollo 11 moon landing was a fabricated event, filmed on a large Hollywood set. According to this theory, Neil Armstrong did not actually step on the moon but was merely acting in a studio. This is purported to have been a staged event by the United States during the Cold War to gain an advantage in the space race against the Soviet Union. Proponents of this idea cite various pieces of "evidence," such as the American flag appearing to wave in the vacuum of space, and inconsistencies in the direction of shadows, which they claim do not align with actual sunlight. It is also suggested that NASA has attempted to conceal this alleged truth.
